<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/nlp-zurich/02-text-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

## Install dependencies

In [1]:
! pip install datasets transformers sentencepiece
!apt install git-lfs
!pip install sklearn


     |████████████████████████████████| 306 kB 20.2 MB/s 
     |████████████████████████████████| 3.4 MB 64.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.7 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 133 kB 66.9 MB/s 
     |████████████████████████████████| 243 kB 65.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.4 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 3.3 MB 65.9 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 271 kB 54.6 MB/s 
     |████████████████████████████████| 160 kB 66.0 MB/s 
     |████████████████████████████████| 192 kB 71.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The

##Fetch and prepare training dataset

In [2]:
%%bash
git clone https://github.com/danielhorizon/lyrics-genreation/
mv lyrics-genreation/genius_data .
rm -r lyrics-genreation

Cloning into 'lyrics-genreation'...
Checking out files: 100% (42/42), done.


In [3]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import re
max_length = 100
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'
test_frac = 0.15

def read_genre(genre):
  return pd.read_csv('genius_data/%s.csv'%genre)[['artist', 'genre', 'title', 'lyrics']]

def build_text_files(df, dest_path):
    text = ''
    for i, row in df.iterrows():
      lyrics = row['lyrics']
      if str(lyrics) == 'nan':
        continue
      text += lyrics + '. '
    text = text.replace('\n\n', '.\n')
    text = re.sub(r'\s', ' ', text)
    text = re.sub(r'\.+', '.', text)    
    text = re.sub(r' +', ' ', text)

    with open(dest_path, 'w') as f:
      f.write(text)
    print(dest_path + ' length: ' + str(len(df)))
    print(str(len(text)) + ' elements')

genres = ['pop','jazz','folk','soul']

df = pd.concat([read_genre(x) for x in genres]).sample(frac=1) # mix different genres

train, test = train_test_split(df, test_size=test_frac)
build_text_files(train, train_path)
build_text_files(test, test_path)

train_dataset.txt length: 19233
25530711 elements
test_dataset.txt length: 3395
4382442 elements


In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling, AutoTokenizer

model_name = "distilgpt2"
model_name_ft = "distilgpt2-finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

!rm cached*
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

rm: cannot remove 'cached*': No such file or directory


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (6568444 > 1024). Running this sequence through the model will result in indexing errors


##Trainer

In [7]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [8]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
batch_size = 32
model = AutoModelWithLMHead.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./" + model_name_ft, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=batch_size, # batch size for training
    per_device_eval_batch_size=batch_size,  # batch size for evaluation
    eval_steps=400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500, # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    push_to_hub=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:787: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
 

Download file pytorch_model.bin:   0%|          | 16.0k/319M [00:00<?, ?B/s]

Download file runs/Jan13_18-14-45_a68d793480cc/1642097694.8503954/events.out.tfevents.1642097694.a68d793480cc.…

Download file runs/Jan13_18-32-01_a68d793480cc/1642098731.0676572/events.out.tfevents.1642098731.a68d793480cc.…

Download file runs/Jan13_18-14-45_a68d793480cc/events.out.tfevents.1642097694.a68d793480cc.74.2:  90%|########…

...

Clean file runs/Jan13_18-32-01_a68d793480cc/events.out.tfevents.1642098731.a68d793480cc.1177.0:  26%|##5      …

Clean file runs/Jan13_18-45-10_a68d793480cc/events.out.tfevents.1642099520.a68d793480cc.1555.0:  26%|##5      …

Clean file runs/Jan13_18-28-00_a68d793480cc/events.out.tfevents.1642098485.a68d793480cc.74.6:  26%|##5       |…

Clean file training_args.bin:  35%|###4      | 1.00k/2.86k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/319M [00:00<?, ?B/s]

In [ ]:
trainer.train()
trainer.save_model()

***** Running training *****
  Num examples = 51315
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4812


Step,Training Loss


In [ ]:
trainer.push_to_hub(commit_message="Training complete!")

In [ ]:
from transformers import pipeline, AutoModelForCausalLM
max_length = 50
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
text_generator_bsl = pipeline("text-generation", 
                              max_length=max_length,
#                              tokenizer=tokenizer, 
#                              model=model,
                              device=0) # device >= 0 places the model on the GPU
text_generation = pipeline('text-generation', 
                           model="./" + model_name_ft, 
                           tokenizer=tokenizer) #,                           config={'max_length':max_length}


Upoload data

In [ ]:
from google.colab import files

uploaded = files.upload()

lines = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'r') as f:
    lines.extend(f.readlines())

##Compute and output results

In [ ]:
from datetime import datetime

now = datetime.now() # current date and time
date_time = now.strftime("%m%d%Y-%H%M%S")

print("date and time:",date_time)	
gen_dict = {'bsl' : text_generator_bsl, 'ft': text_generation}

for gen_name, generator in gen_dict.items():
  filename = 'Results-%s_%s.txt'%(gen_name, date_time)
  results = generator([line.strip() for line in lines])
  with open(file=filename, mode='w') as f:
    for result in results:
      text = result[0]['generated_text'].replace("\n", "")
      f.write(text + '\n')
  files.download(filename)